In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.contrib import layers

In [2]:
# 트레이닝 데이터 로드
x_data = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_activity.csv').sort_values(by='acc_id', ascending=True)
y_data = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_label.csv').sort_values(by='acc_id', ascending=True)

In [3]:
x_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440323 entries, 427043 to 315904
Data columns (total 38 columns):
wk                          440323 non-null int64
acc_id                      440323 non-null object
cnt_dt                      440323 non-null int64
play_time                   440323 non-null float64
npc_exp                     440323 non-null float64
npc_hongmun                 440323 non-null float64
quest_exp                   440323 non-null float64
quest_hongmun               440323 non-null float64
item_hongmun                440323 non-null float64
game_combat_time            440323 non-null float64
get_money                   440323 non-null float64
duel_cnt                    440323 non-null float64
duel_win                    440323 non-null float64
partybattle_cnt             440323 non-null float64
partybattle_win             440323 non-null float64
cnt_enter_inzone_solo       440323 non-null float64
cnt_enter_inzone_light      440323 non-null float64
cnt_e

In [4]:
x_data = x_data[x_data['wk'] == 8].values[:,2:]
y_data = y_data.label.values

In [5]:
x_data = np.concatenate((x_data, x_data**2, x_data**3), axis=1)

In [6]:
x_data = (x_data - x_data.min(axis=0)) / (x_data.max(axis=0) - x_data.min(axis=0))

In [7]:
x_data.shape

(100000, 108)

In [8]:
#트레이닝셋, 발리데이션셋 구분
x_train = x_data[0:int(0.7 * len(x_data)), 2:] 
y_train = y_data[0:int(0.7 * len(x_data))]

x_cv = x_data[int(0.7 * len(x_data)):, 2:] 
y_cv = y_data[int(0.7 * len(x_data)):]

In [9]:
forest = RandomForestClassifier(n_estimators=50, random_state=0, n_jobs=-1, max_features=10, max_depth=20)

In [10]:
forest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [11]:
print("훈련 세트 정확도: {:.3f}".format(forest.score(x_train, y_train)))

훈련 세트 정확도: 0.788


In [12]:
print("크로스발리데이션 세트 정확도: {:.3f}".format(forest.score(x_cv, y_cv)))

크로스발리데이션 세트 정확도: 0.691


In [13]:
#한국어로 된 레이블을 숫자로 인코딩
idx2label = list(set(y_train))
label2idx = {c:i for i,c in enumerate(idx2label)}

#new y_train & y_cv
y_train = [label2idx[v] for v in y_train]
y_cv = [label2idx[v] for v in y_cv]

In [14]:
# 레이블 개수
num_feature = len(x_train[0])
learning_rate = 0.01
training_epoch = 100
batch_size = 100
label = len(idx2label)

In [15]:
X = tf.placeholder(tf.float32, [None, num_feature])
Y = tf.placeholder(tf.int32, [None])
keep_prob = tf.placeholder(tf.float32)

In [16]:
#Y_one_hot
Y_one_hot = tf.one_hot(Y, label)

In [17]:
#Layer1
with tf.name_scope('layer1') as scope:
    w1 = tf.get_variable("weight1", [num_feature, 64], tf.float32, layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([64]), name = "bias1")
    
    layer1 = tf.nn.relu(tf.matmul(X,w1) + b1)
    layer1 = tf.nn.dropout(layer1, keep_prob)

In [18]:
#Layer2
with tf.name_scope('layer2') as scope:
    w2 = tf.get_variable("weight2", [64, 32], tf.float32, layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([32]), name = "bias2")
    
    layer2 = tf.nn.relu(tf.matmul(layer1,w2) + b2)
    layer2 = tf.nn.dropout(layer2, keep_prob)

In [19]:
#Layer3
with tf.name_scope('layer3') as scope:
    w3 = tf.get_variable("weight3", [32, 16], tf.float32, layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([16]), name = "bias3")
    
    layer3 = tf.nn.relu(tf.matmul(layer2,w3) + b3)
    layer3 = tf.nn.dropout(layer3, keep_prob)

In [20]:
#Layer4
with tf.name_scope('layer4') as scope:
    w4 = tf.get_variable("weight4", [16, 8], tf.float32, layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([8]), name = "bias4")
    
    layer4 = tf.nn.relu(tf.matmul(layer3,w4) + b4)
    layer4 = tf.nn.dropout(layer4, keep_prob)

In [21]:
#hypothesis
with tf.name_scope('hypothesis') as scope:
    w5 = tf.get_variable("weight5", [8, label], tf.float32, layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([label]), name = "bias5")
    
    logits = tf.matmul(layer4, w5) + b5

In [22]:
#Cost Function - softmax cross entropy
with tf.name_scope("cost") as scope:
    cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_one_hot)
    cost = tf.reduce_mean(cost)
    tf.summary.scalar("cost", cost)

In [23]:
#Train - Gradient Descent
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(cost)

In [24]:
#Prediction
prediction = tf.argmax(logits, -1)

In [25]:
#Accuracy
with tf.name_scope("accuracy") as scope:
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, tf.argmax(Y_one_hot,-1)), tf.float32))
    tf.summary.scalar("accuracy", accuracy)

In [26]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epoch):
        avg_cost = 0
        total_batch = int(len(x_train) / batch_size)
        
        for i in range(total_batch):
            batch_x = x_train[i*batch_size:(i+1)*batch_size]
            batch_y = y_train[i*batch_size:(i+1)*batch_size]
            
            for i in range(5):
                cost_val, _ = sess.run([cost, train], feed_dict={X:batch_x,Y:batch_y, keep_prob:0.7})
            avg_cost += cost_val/total_batch
        
        print("[epoch : {}]  cost {}".format(epoch, avg_cost))
    
    print("accuracy : {}".format(sess.run(accuracy, feed_dict={X:x_train, Y:y_train, keep_prob:1})))

[epoch : 0]  cost 1.4154248598643706
[epoch : 1]  cost 1.3894473896707802
[epoch : 2]  cost 1.3867077159881571
[epoch : 3]  cost 1.3859517688410636
[epoch : 4]  cost 1.385778485706875
[epoch : 5]  cost 1.3855467840603428
[epoch : 6]  cost 1.384480219738823
[epoch : 7]  cost 1.3833102124077936
[epoch : 8]  cost 1.3774863507066466
[epoch : 9]  cost 1.361754185812814
[epoch : 10]  cost 1.3466261913095223
[epoch : 11]  cost 1.3310263659272883
[epoch : 12]  cost 1.3091901436873847
[epoch : 13]  cost 1.2839823440143034
[epoch : 14]  cost 1.2644122374057758
[epoch : 15]  cost 1.250306055375508
[epoch : 16]  cost 1.2360928225517271
[epoch : 17]  cost 1.2213616372857774
[epoch : 18]  cost 1.2046490081718984
[epoch : 19]  cost 1.1910372920547212
[epoch : 20]  cost 1.175767372846603
[epoch : 21]  cost 1.167258646403039
[epoch : 22]  cost 1.1586048700128282
[epoch : 23]  cost 1.1543113952023634
[epoch : 24]  cost 1.1480720460414913
[epoch : 25]  cost 1.1461317828723372
[epoch : 26]  cost 1.1437288

In [27]:
tf.reset_default_graph()
